<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/2000px-TensorFlowLogo.svg.png" width="500"/>

In [4]:
import tensorflow as tf
import numpy as np

## Variables

<img src="http://colah.github.io/posts/2015-08-Backprop/img/tree-def.png" width="500">

In [7]:
tf.reset_default_graph() #reiniciar grafos

#Inicializar grafos
x = tf.Variable(3, name="x") #Definir variable del grafo
y = tf.Variable(4, name="y") #Definir variable del grafo
z = x + y #Definir operaciones
f = x*x*y + y - z + 1 #Definir operaciones

## Eval vs Run

In [8]:
with tf.Session() as sess: #Sesiones para evaluar los grafos
    x.initializer.run() #inicializar variable
    y.initializer.run() #inicializar variable
    resultz = z.eval() #evaluar resultado
    resultf1 = f.eval() #evaluar resultado
    resultf2 = sess.run(f) #evaluar resultado
    print('z =',resultz)
    print('f =',resultf1)
    print('f =',resultf2)

z = 7
f = 34
f = 34


In [11]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    resultf,resultz = sess.run([f,z],feed_dict={x:1,y:1,z:1})
    print('z =',resultz)
    print('f =',resultf)

z = 1
f = 2


# Gradiente Descendente

<img src="https://www.oreilly.com/library/view/hands-on-machine-learning/9781491962282/assets/mlst_0402.png" width="500"/>

In [5]:
tf.reset_default_graph()
valx=3
x = tf.Variable(valx,dtype=tf.float32)
f=x**2
df=2*x

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for i in range(1000):
        resf,resd=sess.run([f,df],feed_dict={x:valx})
        valx-=0.01*resd
    print(valx)

5.04890206182e-09


## Autodiff

In [7]:
tf.reset_default_graph()
valx=3
valy=-3
x = tf.Variable(valx,dtype=tf.float32)
y = tf.Variable(valy,dtype=tf.float32)
f=(x-5)**2+(y+5)**2
df=tf.gradients(f, [x,y])

In [8]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for i in range(1000):
        resf,resd=sess.run([f,df],feed_dict={x:valx,y:valy})
        valx-=0.01*resd[0]
        valy-=0.01*resd[1]
    print(valx,valy)

4.99999977112 -4.99999977112


# Regresión Lineal

<img src="https://i.ytimg.com/vi/zPG4NjIkCjc/maxresdefault.jpg" width="700" />

In [9]:
m=100
xr=np.linspace(0,1,m).reshape(m,1)
yr=3*xr+5+np.random.rand(m,1)
xr=np.c_[np.ones([m,1]),xr]

## Ecuación Normal

In [22]:
w=np.matmul(np.matmul(np.linalg.inv(np.matmul(xr.T, xr)), xr.T), yr)
print(w)

[[ 5.46608307]
 [ 3.0589149 ]]


## Gradiente Descendente

In [11]:
tf.reset_default_graph()

X = tf.constant(xr, dtype=tf.float32)
y = tf.constant(yr, dtype=tf.float32)

theta = tf.Variable(tf.random_uniform([2, 1]))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))

In [12]:
learning_rate = 0.01
grads = tf.gradients(mse, theta)[0]
training_op = tf.assign(theta, theta - learning_rate * grads) #Gradiente Descendente

In [23]:
n_epochs = 1000

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        _,cur_mse=sess.run([training_op,mse])
        if epoch % 100 == 0: print("Epoca", epoch, "MSE =", cur_mse)
            
    print(theta.eval())

Epoca 0 MSE = 35.1234
Epoca 100 MSE = 0.301343
Epoca 200 MSE = 0.096766
Epoca 300 MSE = 0.0955298
Epoca 400 MSE = 0.0954961
Epoca 500 MSE = 0.0954758
Epoca 600 MSE = 0.0954603
Epoca 700 MSE = 0.0954483
Epoca 800 MSE = 0.0954393
Epoca 900 MSE = 0.0954323
[[ 5.45850182]
 [ 3.07305932]]


## GradientDescentOptimizer

In [24]:
tf.reset_default_graph()

X = tf.constant(xr, dtype=tf.float32)
y = tf.constant(yr, dtype=tf.float32)

theta = tf.Variable(tf.random_uniform([2, 1]))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))

In [25]:
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) #Optimizador
training_op = optimizer.minimize(mse)                                      #Objetivo del optimizador

In [26]:
n_epochs = 1000

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        _,cur_mse=sess.run([training_op,mse])
        if epoch % 100 == 0: print("Epoca", epoch, "MSE =", cur_mse)
            
    print(theta.eval())

Epoca 0 MSE = 40.4498
Epoca 100 MSE = 0.351307
Epoca 200 MSE = 0.111342
Epoca 300 MSE = 0.106536
Epoca 400 MSE = 0.103911
Epoca 500 MSE = 0.10191
Epoca 600 MSE = 0.10038
Epoca 700 MSE = 0.0992101
Epoca 800 MSE = 0.0983155
Epoca 900 MSE = 0.0976316
[[ 5.39089203]
 [ 3.19921422]]


## Adam

In [27]:
tf.reset_default_graph()

X = tf.constant(xr, dtype=tf.float32)
y = tf.constant(yr, dtype=tf.float32)

theta = tf.Variable(tf.random_uniform([2, 1]))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))

In [43]:
learning_rate = 0.1
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) #Optimizador
training_op = optimizer.minimize(mse)                           #Objetivo del optimizador

In [44]:
n_epochs = 1000

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        _,cur_mse=sess.run([training_op,mse])
        eval_current_lr = sess.run(current_lr)
        if epoch % 100 == 0: print("Epoca", epoch, "MSE =", cur_mse)
            
    print(theta.eval())

Epoca 0 MSE = 36.4757
Epoca 100 MSE = 0.208683
Epoca 200 MSE = 0.128353
Epoca 300 MSE = 0.101626
Epoca 400 MSE = 0.0961948
Epoca 500 MSE = 0.0954768
Epoca 600 MSE = 0.0954136
Epoca 700 MSE = 0.0954099
Epoca 800 MSE = 0.0954098
Epoca 900 MSE = 0.0954098
[[ 5.46607733]
 [ 3.05892396]]


# Mini-Batch

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(shape=(None,2), dtype=tf.float32)#Placeholder en vez de Constant
y = tf.placeholder(shape=(None,1), dtype=tf.float32)#Placeholder en vez de Constant

In [ ]:
theta = tf.Variable(tf.random_uniform([2, 1]))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))

In [ ]:
learning_rate = 0.1
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)#Optimizador
training_op = optimizer.minimize(mse)                          #Objetivo

In [ ]:
batch_size = 32
n_batches = m//batch_size
n_epochs = 1000

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        indices=np.arange(m)
        np.random.shuffle(indices)
        for batch in range(n_batches):
            X_batch=xr[indices[batch*batch_size:(batch+1)*batch_size],:]
            y_batch=yr[indices[batch*batch_size:(batch+1)*batch_size],:]
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        if epoch % 100 == 0: print("Epoca", epoch, "MSE =", mse.eval(feed_dict={X:xr,y:yr}))
            
    print(theta.eval())

# Guardar y Restaurar

In [ ]:
tf.reset_default_graph()

X = tf.constant(xr, dtype=tf.float32)
y = tf.constant(yr, dtype=tf.float32)

theta = tf.Variable(tf.random_uniform([2, 1]),name="theta")
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error))

learning_rate = 0.1
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()  #Grabador

In [ ]:
n_epochs = 1000
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        _,cur_mse=sess.run([training_op,mse])
        if epoch % 100 == 0: print("Epoca", epoch, "MSE =", cur_mse)
        save_path = saver.save(sess, "/tmp/my_model.ckpt")           #Guardar Checkpoints
    print(theta.eval())
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")         #Guardar modelo final

## Restaurar

### Error

Restaurar con un grafo diferente

In [ ]:
tf.reset_default_graph()
theta = tf.Variable(tf.random_uniform([2, 2]))
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    print(theta.eval())

## Sin Error

Restaurar con el mismo grafo

In [ ]:
tf.reset_default_graph()

X = tf.constant(xr, dtype=tf.float32)
y = tf.constant(yr, dtype=tf.float32)

theta = tf.Variable(tf.random_uniform([2, 1]))
y_pred = tf.matmul(X, theta)
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

learning_rate = 0.1
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    print(theta.eval())

### Graph

Restaurar grafo

In [ ]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    print(theta.eval())